<a href="https://colab.research.google.com/github/QuantumNews/Grovers-search-algorithm-and-QAE/blob/main2/Gover's_algorithm_and_QAE_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip3 install qiskit

     |████████████████████████████████| 6.5 MB 4.5 MB/s 
     |████████████████████████████████| 18.0 MB 90 kB/s 
     |████████████████████████████████| 238 kB 70.8 MB/s 
     |████████████████████████████████| 200 kB 63.2 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 1.6 MB 48.4 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 
     |████████████████████████████████| 37.5 MB 1.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 943 kB 55.9 MB/s 
     |████████████████████████████████| 3.6 MB 44.9 MB/s 
     |████████████████████████████████| 113 kB 51.8 MB/s 
  Created wheel for qiskit: filename=qiskit-0.35.0-py3-none-any.whl size=11839 sha256=aeca4123b290c824f8b9611c52ef7745221f14044c89261f47264c96c1307c01
  Stored in directory: /root/.cache/pip/wheels/1e/0a/42/93e5cc11795fb0a85fcf6686eac2791def0c9a1cad0a774

First instal Qiskit (above)
then import some useful packages from QIskit (for circuit building and access backend) as well as numpy and pyplot for some mathemtics and visualization:

In [ ]:
import qiskit
import matplotlib.pyplot as plt
import matplotlib.axes as axes
%matplotlib inline
import numpy as np
import networkx as nx
from math import pi
from qiskit import Aer
from qiskit.tools.visualization import plot_histogram
from qiskit.circuit.library import TwoLocal

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *


backend = BasicAer.get_backend('unitary_simulator')

In [ ]:
from qiskit import *

In [ ]:
Here my API token gets loaded to give us access:

In [ ]:
IBMQ.save_account('63af478871059f44c837562334bb53037b1591a9140852e357e30b6d30617cb79596f7e6835d571b3374e295446d6222be23ab1ac84d45817eb91a22a23c4ddd')

In [ ]:
overwrite=True


In [ ]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [ ]:
from qiskit import *


In [ ]:
from qiskit.providers.ibmq.exceptions import IBMQAccountCredentialsInvalidToken

Lets take a look at a how Grovers seach algorithm works. 

If you have a list , classically the complexity depends on how many times you need to call the oracle, classically this means atleast N/2  to N. 

However, in the case of Grovers search algorithm, we can reduce the complexity scaling to O(SQRT_N). This is the quadratic speedup promised by QAE.

Grover's aalgorithm has two main components. the first is the oracle, which qeuaries an input state to assertain whether it is the required or expected value, and secondly a reflection operator. The reflection operator is the compoenet that amplifies the amplitude of a specific state:



The oracle:
This is some fucntion/gate that flips the sign of the winning state so that we can identify when we have found it. 
so for exsample, if we have two qubits, we can have the following combinations of vector outputs:

00
01
10
11

and oracle acting on this vector scpae would leave the same except for the final state (11), which would be flipped to give (-11). 
Luckly this is simple for a 2 qubit system where the control Z gate would do just this:

In [ ]:
#define the oracle circuit
oracle = QuantumCircuit(2,name ='oracle')
oracle.cz(0,1)
oracle.to_gate()
oracle.draw()

q_0: ─■─
      │ 
q_1: ─■─

SO lets check this by simply adding a superposition with a Haddamard gate, followed by the CZ gate (oracle):

In [ ]:
backend = Aer.get_backend('statevector_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle,[0,1])

In [ ]:
job = execute(grover_circ,backend)
result = job.result()


In [ ]:
sv = result.get_statevector()
np.around(sv,2)

array([ 0.5+0.j,  0.5+0.j,  0.5+0.j, -0.5+0.j])

As can be seen above, the final result (i.e. vector output: 11) have been flipped (negative sign). 

In [ ]:
simulator = Aer.get_backend('qasm_simulator')
result = execute(grover_circ, backend = simulator).result()
from qiskit.tools.visualization import plot_histogram
plot_histogram(result.get_counts(grover_circ))

QiskitError: ignored

So we have seen that the oracle function works for our simoe 2 qubit circuit... now lets add the reflection operator so that we can perform amplitude amplification.

As mentioned above, the reflection operator is a function/gate that acts on a specified state and tries to bringin it closer to the winner/expected state. 

In [ ]:
reflection = QuantumCircuit(2,name= 'reflection')
reflection.h([0,1])
reflection.z([0,1])
reflection.cz(0,1)
reflection.h([0,1])
reflection.to_gate()

Instruction(name='reflection', num_qubits=2, num_clbits=0, params=[])

In [ ]:
reflection.draw()


┌───┐┌───┐   ┌───┐
q_0: ┤ H ├┤ Z ├─■─┤ H ├
     ├───┤├───┤ │ ├───┤
q_1: ┤ H ├┤ Z ├─■─┤ H ├
     └───┘└───┘   └───┘

In [ ]:
backend = Aer.get_backend('qasm_simulator')
grover_circ = QuantumCircuit(2,2)
grover_circ.h([0,1])
grover_circ.append(oracle,[0,1])
grover_circ.append(reflection,[0,1])
grover_circ.measure([0,1],[0,1])

In [ ]:
grover_circ.draw()

┌───┐┌─────────┐┌─────────────┐┌─┐   
q_0: ┤ H ├┤0        ├┤0            ├┤M├───
     ├───┤│  oracle ││  reflection │└╥┘┌─┐
q_1: ┤ H ├┤1        ├┤1            ├─╫─┤M├
     └───┘└─────────┘└─────────────┘ ║ └╥┘
c: 2/════════════════════════════════╩══╩═
                                     0  1

In [ ]:
job=execute(grover_circ,backend,shots=1)
result=job.result()
result.get_counts()

{'11': 1}

As we see, we get the expected winner state 11 with just a single shot! remember that the complextity scales like O(sqrt_N).